<a href="https://colab.research.google.com/github/BobTheConstructor/Estudos/blob/main/PyDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o faker, Gerador de codigo ficticios

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.4 MB/s eta 0:00:00


importação das bibliotecas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from faker import Faker
from datetime import datetime, timedelta

Criação dos dados aleatorios e do DataFrame aqui chamado de "tabela"

In [4]:
fake = Faker('pt_BR')
Faker.seed(123)
random.seed(123)

# Função para gerar medida do pneu
def gerar_medida(veiculo):
    if veiculo == 'carro':
        largura = random.choice([165, 175, 185, 195])
        altura = random.choice([65, 75])
        aro = random.choice([13, 14, 15, 16])
    else:  # caminhão
        largura = random.choice([205, 215, 225])
        altura = random.choice([45, 55, 60, 65, 70])
        aro = random.choice([15, 16, 17])
    return f"{largura}/{altura}.{aro}"

cnpj = []
for _ in range(10):
  x = fake.cnpj()
  cnpj.append(x)

# Geração dos dados
dados = []
for _ in range(200):
    empresa = random.choice(['Sim', 'Não'])
    cpf_cnpj = random.choice(cnpj) if empresa == 'Sim' else fake.cpf()
    data_compra = fake.date_between(start_date='-2y', end_date='today')
    veiculo = random.choice(['carro', 'caminhao'])
    medida_pneu = gerar_medida(veiculo)
    remode = random.choice(['Sim', 'Não'])
    quantidade = random.randint(1, 6)

    if remode == 'Sim':
        valor_unitario = round(random.uniform(150.00, 190.00), 2)
    else:
        valor_unitario = round(random.uniform(200.00, 250.00), 2)

    valor_total = round(quantidade * valor_unitario, 2)

    dados.append([
        empresa,
        cpf_cnpj,
        data_compra,
        veiculo,
        medida_pneu,
        remode,
        quantidade,
        valor_unitario,
        valor_total
    ])

# Criar DataFrame
colunas = [
    'Empresa',
    'cpf_cnpj',
    'data_compra',
    'veiculo',
    'medida_pneu',
    'Remolde',
    'Qnt_vendas',
    'Vr_unitario',
    'Vr_Total'
]

tabela = pd.DataFrame(dados, columns=colunas)

# Corrigindo a data para dd/mm/yyyy
tabela['data_compra'] = pd.to_datetime(tabela['data_compra']).dt.strftime('%d/%m/%Y')



Um pequeno teste para se ter uma noção se a tabela corresponde as espectativas

In [6]:
tabela.head()

,Empresa,cpf_cnpj,data_compra,veiculo,medida_pneu,Remolde,Qnt_vendas,Vr_unitario,Vr_Total
0,Sim,12.860.734/0001-20,05/02/2024,carro,195/75.13,Sim,4,171.45,685.80
1,Não,536.470.219-70,16/06/2024,caminhao,205/55.15,Não,5,216.69,1083.45
2,Sim,85.310.792/0001-53,01/05/2025,carro,195/65.16,Sim,1,162.62,162.62
3,Não,287.360.451-44,04/10/2023,carro,165/65.14,Sim,1,161.67,161.67
4,Não,459.386.072-56,23/05/2024,caminhao,215/65.15,Não,3,226.08,678.24


Media do total de vendas


In [59]:
md = tabela['Vr_Total'].mean(axis=0).round(2)
mdVendas = tabela['Qnt_vendas'].mean(axis=0)
print(f'A media de vendas totais é: {md}, tendo em vista que cada cliente compra: {mdVendas} Pneus em media')

A media de vendas totais é: 651.93, tendo em vista que cada cliente compra: 3.28 Pneus em media


A maior compra de cada cnpj filtradas da maior para a menor

In [69]:
tabela.loc[tabela[tabela['Empresa'] == 'Sim'].groupby('cpf_cnpj')['Vr_Total'].idxmax()].sort_values(by="Vr_Total", ascending=False)

,Empresa,cpf_cnpj,data_compra,veiculo,medida_pneu,Remolde,Qnt_vendas,Vr_unitario,Vr_Total
79,Sim,74.960.283/0001-50,20/06/2024,caminhao,215/55.17,Não,6,238.43,1430.58
95,Sim,73.508.492/0001-02,17/02/2025,carro,195/65.15,Não,6,231.63,1389.78
114,Sim,87.624.019/0001-97,03/10/2024,carro,175/65.15,Não,6,230.58,1383.48
32,Sim,76.501.482/0001-25,30/05/2024,carro,195/75.13,Não,6,219.33,1315.98
69,Sim,17.492.530/0001-05,26/06/2023,carro,165/65.13,Não,6,212.31,1273.86
197,Sim,85.310.792/0001-53,01/07/2023,caminhao,215/65.17,Não,5,214.12,1070.60
174,Sim,61.025.374/0001-56,08/09/2023,caminhao,225/60.17,Não,5,208.11,1040.55
86,Sim,68.520.143/0001-68,09/12/2023,caminhao,225/65.17,Não,4,235.73,942.92
66,Sim,04.163.295/0001-78,10/09/2023,caminhao,225/65.17,Não,3,233.10,699.30
0,Sim,12.860.734/0001-20,05/02/2024,carro,195/75.13,Sim,4,171.45,685.80
